In [1]:
from package import *

import requests
from pprint import pprint
from functools import partial
import pandas
import matplotlib.pyplot as plt
import os
import sys

github_folder = os.path.join(os.getenv('USERPROFILE'), 'Documents', 'Github', 'python-youtube-api')
sys.path.append(github_folder)

pprint = partial(pprint, width = 200)

#youtube = YouTubeDatabase(github.youtube_api_key, 'youtube_database')
killian = "UCjdQaSJCYS4o2eG93MvIwqg"
test_video = "836Lv_swaCk"
roosterteeth = 'UCzH3iADRIq1IJlIXjfNgTpA'
vs_playlist = "PLTur7oukosPH7Kvex9-rUiZgVIVGt6C--"
youtube = YouTube(github.youtube_api_key)
print("Finished!")


Finished!


In [2]:
funhaus_id = "UCboMX_UNgaPBsUOIgasn3-Q"
vs = youtube.getChannelItems(killian)
vids = youtube.getVideos(vs.items)
print(vs)

ListResource('"_gJQceDMxJ8gP-8T2HLXUoURK8c/qRCTIuJ2Ffc1UM83vbYqEB8dSxI"', 86)


In [3]:
vs.summary()
vids.summary()

ListResource('"_gJQceDMxJ8gP-8T2HLXUoURK8c/qRCTIuJ2Ffc1UM83vbYqEB8dSxI"', 86)
itemCount:  86
	 PlaylistItemResource('UUjdQaSJCYS4o2eG93MvIwqg', 'Kilian Experience', 'The Extremely Overpowered Dwarf')
	 PlaylistItemResource('UUjdQaSJCYS4o2eG93MvIwqg', 'Kilian Experience', 'Kilians Game Of The Year 2017')
	 PlaylistItemResource('UUjdQaSJCYS4o2eG93MvIwqg', 'Kilian Experience', 'Garbage Guide To Pokemon Platinum')
	 PlaylistItemResource('UUjdQaSJCYS4o2eG93MvIwqg', 'Kilian Experience', 'Is Shadow The Hedgehog The Best Game Ever?')
	 PlaylistItemResource('UUjdQaSJCYS4o2eG93MvIwqg', 'Kilian Experience', 'Greatest Europa Universalis 4 Challenge')
	 PlaylistItemResource('UUjdQaSJCYS4o2eG93MvIwqg', 'Kilian Experience', 'Dumbest Fortnite Battle Royale Player')
	 PlaylistItemResource('UUjdQaSJCYS4o2eG93MvIwqg', 'Kilian Experience', 'Garbage Retrospective Of Nintendo Entertainment System')
	 PlaylistItemResource('UUjdQaSJCYS4o2eG93MvIwqg', 'Kilian Experience', 'The Most Powerful Orc')
	 PlaylistIte

In [4]:
pprint(vs.items[0].content_details)

{}


In [6]:
print(PlaylistItemResource(vs.items[0].response))

PlaylistItemResource('None', 'None', 'None')
